## Fasttext

following tutorial https://fasttext.cc/docs/en/supervised-tutorial.html

In [ ]:
import fasttext

In [ ]:
!wc ./fasttext/cooking.stackexchange.txt
#splitiing dataset into training and validation sets
!head -n 12404 ./fasttext/cooking.stackexchange.txt > ./fasttext/cooking.train
!tail -n 3000 ./fasttext/cooking.stackexchange.txt > ./fasttext/cooking.valid


In [ ]:
#our first classifier
model = fasttext.train_supervised(input="./fasttext/cooking.train")
#saving the model
model.save_model("./fasttext/model_cooking.bin")

In [ ]:
model.predict("Which baking dish is best to bake a banana bread ?")

In [ ]:
model.predict("Why not put knives in the dishwasher?")

In [ ]:
model.test("./fasttext/cooking.valid")

In [ ]:
model.test("./fasttext/cooking.valid", k=5)

In [ ]:
model.predict("Why not put knives in the dishwasher?", k=5)

## TODO
- prepare an input file with categories and their keywords
- split dataset in training and validation
- run simple model settings (defaukt as above)
 - feed GOOOGLE search trands!

 
- try to use it as a word vectorizer and measure distance?

In [ ]:
## load the data
import sys
# Add src folder to the path
sys.path.append('../src/')
from data.preprocess_keywords import make_cleaned_keywords_df
from data.make_datasets import get_data
# Specify wanted time range
start_date = '2017-01-01'
end_date = '2022-01-01'
# Path to data
data_file = '/home/anya_m/data/dw-project-data/CMS_2010_to_June_2022_ENGLISH.json'
# Load and extract data within time range
df_subset = get_data(data_file, start_date, end_date)
# Cleans keywords and saves data as a dataframe
make_cleaned_keywords_df(df_subset, start_date, end_date)

In [296]:
# Load DW data
import pandas as pd
start_date = '2017-01-01' #'2019-01-01' #'2021-01-01'
end_date = '2022-01-01'
filepath = '../data/interim/clean_keywords_' + start_date + '_' + end_date + '.json'
df = pd.read_json(filepath, orient ='split', compression = 'infer')

In [298]:
display(len(df))
len(df_subset)

69432

69432

In [237]:
# Load Google data
df_google = pd.read_json('../data/interim/2021-daily-trending-searches.json', orient ='split', compression = 'infer')

In [238]:
# Keeps only google data within DW data date range
start_date_dw=start_date
end_date_dw=end_date
df_google.sort_values(by ='date', inplace = True) 
mask = (pd.to_datetime(df_google['date']) > start_date_dw) & (pd.to_datetime(df_google['date']) <= end_date_dw)
df_google_subset = df_google.loc[mask].copy()
google_topic=df_google_subset['topic_title']
google_topic_type=df_google_subset['topic_title']

In [ ]:
df_google_subset.head()

In [299]:
#add column teaser to the df cleaned dataframe
df=df.merge(df_subset['teaser'], left_on=df_subset['id'], right_on=df['id'])
df=df.drop(['key_0'], axis=1)
df=df.merge(df_subset['text'], left_on=df['id'], right_on=df_subset['id'] )

In [297]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69432 entries, 0 to 69431
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            69432 non-null  int64         
 1   lastModifiedDate              69432 non-null  object        
 2   Date                          69432 non-null  datetime64[ns]
 3   keywordStrings                69432 non-null  object        
 4   keywordStringsCleanAfterFuzz  69432 non-null  object        
 5   cleanFocusCategory            68481 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 3.7+ MB


In [241]:
df_clean=df.dropna(how='any')
df_clean_copy = df_clean.copy(deep=True)
##TODO
#df_clean['cleanFocusCategory']=='Law and Justice' replace with Law-and-Justice  and other categories

df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Law and Justice'], 'Law-and-Justice')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Nature and Environment'], 'Nature-and-Environment')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Human Rights'], 'Human-Rights')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Cars and Transportation'], 'Cars-and-Transportation')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Press Freedom'], 'Press-Freedom')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Digital World'], 'Digital-World')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Rule of Law'], 'Rule-of-Law')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Learning German'], 'Learning-German')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Freedom of Speech'], 'Freedom-of-Speech')

/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [242]:
df_clean['cleanFocusCategory'].unique()

array(['History', 'Politics', 'Society', 'Culture', 'Media',
       'Law-and-Justice', 'Catastrophe', 'Lifestyle', 'Business',
       'Travel', 'Sports', 'Science', 'Health', 'Nature-and-Environment',
       'Cars-and-Transportation', 'Education', 'Religion', 'Human-Rights',
       'Technology', 'Learning-German', 'Digital-World', 'Migration',
       'Offbeat', 'Innovation'], dtype=object)

In [243]:
label=df_clean['cleanFocusCategory'].apply(lambda x: ('__label__' +x+' '))
kw=df_clean['keywordStrings'].apply(lambda x: (', '.join(x)))
teaser=df_clean['teaser']

In [ ]:
print(label+teaser)

In [ ]:
print(label+kw)

In [244]:
from tqdm import tqdm
!rm ./fasttext/teaser.txt
with open('./fasttext/teaser.txt', 'a') as f:
    for i, el in tqdm(enumerate(label)):
        #print(i)
        #print(kw[i])
        print(el+teaser.iloc[i],file=f)
    
    
    #el.apply(lambda x: ' '.join(kw[i]))

68481it [00:01, 50816.63it/s]


In [245]:
from tqdm import tqdm
!rm ./fasttext/output.txt
with open('./fasttext/output.txt', 'a') as f:
    for i, el in tqdm(enumerate(label)):
        #print(i)
        #print(kw[i])
        print(el+kw.iloc[i],file=f)
    
    
    #el.apply(lambda x: ' '.join(kw[i]))

68481it [00:00, 71646.11it/s]


### Trying Fasttext
- DW keywords + DW categories  - training + validation
- DW teaser + DW cats          - training + validation
- DW teaser / keywords for training , google for prediction
- - validation based on similarity of google to DW keywords (rapidfuzz? cdist / word embeddings)
- - make graphics

- DW text for training, google for prediction
- - clean html (beautiful soup?)

In [246]:
import fasttext

In [247]:
#splitiing dataset into training and validation sets

!head -n 50000 ./fasttext/output.txt > ./fasttext/output.train
!tail -n 20000 ./fasttext/output.txt > ./fasttext/output.valid

In [248]:
#splitiing dataset into training and validation sets
!cat ./fasttext/output.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/output.preprocessed.txt

!head -n 50000 ./fasttext/output.preprocessed.txt > ./fasttext/output_cl.train
!tail -n 20000 ./fasttext/output.preprocessed.txt > ./fasttext/output_cl.valid

In [268]:
#our first classifier
train_file="./fasttext/output.train"
model = fasttext.train_supervised(input=train_file, lr=1.0, epoch=12)
#saving the model
model.save_model("./fasttext/model_output.bin")

Read 0M words
Number of words:  46227
Number of labels: 22
Progress: 100.0% words/sec/thread:  445314 lr:  0.000000 avg.loss:  0.647694 ETA:   0h 0m 0s


In [269]:
model.test("./fasttext/output.valid",k=1)

(19964, 0.6350430775395712, 0.6350430775395712)

In [270]:
!rm ./fasttext/model_cleaned_output.bin
#our first classifier
train_file="./fasttext/output_cl.train"
model_cleaned = fasttext.train_supervised(input=train_file, lr=1.0, epoch=12, bucket=200000, dim=50, loss='hs')
#saving the model
model_cleaned.save_model("./fasttext/model_cleaned_output.bin")

Read 0M words
Number of words:  31769
Number of labels: 22
Progress: 100.0% words/sec/thread: 1760339 lr:  0.000000 avg.loss:  0.795935 ETA:   0h 0m 0s


In [271]:
model_cleaned.test("./fasttext/output_cl.valid",k=1)

(19964, 0.6430575035063114, 0.6430575035063114)

In [303]:
#automatic search for best hyperparameters https://fasttext.cc/docs/en/autotune.html 
model_best = fasttext.train_supervised(input=train_file, autotuneValidationFile='./fasttext/output_cl.valid')

Progress: 100.0% Trials:   23 Best score:  0.658736 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  31769
Number of labels: 22
Progress: 100.0% words/sec/thread:  741156 lr:  0.000000 avg.loss:  0.837053 ETA:   0h 0m 0s


In [304]:
model_best.test('./fasttext/output_cl.valid')

(19964, 0.6515728310959727, 0.6515728310959727)

In [272]:
model.predict("house",k=5)

(('__label__Politics',
  '__label__Culture',
  '__label__Business',
  '__label__Law-and-Justice',
  '__label__Sports'),
 array([9.24510300e-01, 4.67729419e-02, 2.85993479e-02, 1.13516013e-04,
        3.87617984e-05]))

In [273]:
model_cleaned.predict("house",k=5)

(('__label__politics',
  '__label__business',
  '__label__culture',
  '__label__travel',
  '__label__lifestyle'),
 array([0.34492129, 0.27110988, 0.25599673, 0.07063578, 0.02948774]))

In [ ]:
#The output are the number of samples (here 14999), 
# the precision at one (0.6968)  69% and 
# the recall at one (0.0541).

#The precision is the number of correct labels among the labels predicted by fastText. 
# The recall is the number of labels that successfully were predicted,
# among all the real labels. 

With a few steps, we were able to go from a precision at one of 12.4% to 59.9%. Important steps included:

- preprocessing the data ;
- changing the number of epochs (using the option -epoch, standard range [5 - 50]) ;
- changing the learning rate (using the option -lr, standard range [0.1 - 1.0]) ;
- using word n-grams (using the option -wordNgrams, standard range [1 - 5]).

In [ ]:
model.predict("knives", k=5)

In [274]:
#model with uncleaned teaser 

#splitiing dataset into training and validation sets

!head -n 50000 ./fasttext/teaser.txt > ./fasttext/teaser.train
!tail -n 20000 ./fasttext/teaser.txt > ./fasttext/teaser.valid

In [277]:
# classifier
train_file="./fasttext/teaser.train"
model_file="./fasttext/model_teaser_output.bin"
test_file="./fasttext/teaser.valid"

model_teaser = fasttext.train_supervised(input=train_file, lr=1.0, epoch=25)
#saving the model
model_teaser.save_model(model_file)
#test model

model_teaser.test(test_file)

Read 1M words
Number of words:  94005
Number of labels: 22
Progress: 100.0% words/sec/thread: 1001291 lr:  0.000000 avg.loss:  0.359905 ETA:   0h 0m 0s


(19927, 0.6028002208059416, 0.6028002208059416)

In [278]:
#model with cleaned teaser 

#splitiing dataset into training and validation sets

!cat ./fasttext/teaser.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/teaser.preprocessed.txt

!head -n 40000 ./fasttext/teaser.preprocessed.txt > ./fasttext/teaser_cl.train
!tail -n 15000 ./fasttext/teaser.preprocessed.txt > ./fasttext/teaser_cl.valid

In [284]:
# classifier with cleaned teaser tested on teaser
train_file="./fasttext/teaser_cl.train"
model_file="./fasttext/model_teaser_cl_output.bin"
test_file="./fasttext/teaser_cl.valid"

model_teaser_cl = fasttext.train_supervised(input=train_file, lr=0.9, epoch=25)
#saving the model
model_teaser_cl.save_model(model_file)
#test model

model_teaser_cl.test(test_file)

Read 1M words
Number of words:  51697
Number of labels: 21
Progress: 100.0% words/sec/thread: 1067246 lr:  0.000000 avg.loss:  0.376231 ETA:   0h 0m 0s


(14794, 0.5892929566040287, 0.5892929566040287)

In [288]:
# classifier with cleaned teaser tested on DW kw
train_file="./fasttext/teaser.preprocessed.txt" #all cleaned teasers +cat
model_file="./fasttext/model_teaser_pred_kw.bin"
test_file="./fasttext/output.preprocessed.txt" #all cleaned dw kw + cat

model_teaser_kw = fasttext.train_supervised(input=train_file, lr=1.0, epoch=12)
#saving the model
model_teaser_kw.save_model(model_file)
#test model

model_teaser_kw.test(test_file)

Read 2M words
Number of words:  66706
Number of labels: 24
Progress: 100.0% words/sec/thread: 1065250 lr:  0.000000 avg.loss:  0.680163 ETA:   0h 0m 0s


(68481, 0.6716899577985135, 0.6716899577985135)

In [ ]:
# classifier with cleaned teaser tested on DW kw
train_file="./fasttext/teaser.preprocessed.txt" #all cleaned teasers +cat
model_file="./fasttext/model_teaser_pred_kw.bin"
test_file="./fasttext/output.preprocessed.txt" #all cleaned dw kw + cat

model_teaser_kw = fasttext.train_supervised(input=train_file, lr=1.0, epoch=50, bucket=200000, dim=50, loss='hs')
#saving the model
model_teaser_kw.save_model(model_file)
#test model

model_teaser_kw.test(test_file)

In [289]:
# classifier with cleaned teaser tested on DW kw
train_file="./fasttext/teaser.preprocessed.txt" #all cleaned teasers +cat
model_file="./fasttext/model_teaser_pred_kw.bin"
test_file="./fasttext/output.preprocessed.txt" #all cleaned dw kw + cat

model_teaser_kw = fasttext.train_supervised(input=train_file, lr=1.0, wordNgrams=5, epoch=50, bucket=200000, dim=50, loss='hs')
#saving the model
model_teaser_kw.save_model(model_file)
#test model

model_teaser_kw.test(test_file)

Read 2M words
Number of words:  66706
Number of labels: 24
Progress: 100.0% words/sec/thread:  582548 lr:  0.000000 avg.loss:  5.385403 ETA:   0h 0m 0s


(68481, 0.625472758867423, 0.625472758867423)

In [ ]:
#google keywords to dw categories


In [ ]:
# # Skipgram model :
# model = fasttext.train_unsupervised('data.txt', model='skipgram')

# # or, cbow model :
# model = fasttext.train_unsupervised('data.txt', model='cbow')


# print(model.words)   # list of words in dictionary
# print(model['king']) # get the vector of the word 'king'